In [3]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('data_text/wiki.model')
model.wv.most_similar(positive=['아빠','여성'], negative=['남성'])

[('엄마', 0.8541994690895081),
 ('아저씨', 0.6801633834838867),
 ('신혼', 0.6701761484146118),
 ('친구', 0.6636468172073364),
 ('할머니', 0.6597239375114441),
 ('어디가', 0.6566115617752075),
 ('아줌마', 0.6550686359405518),
 ('선생님', 0.6481844782829285),
 ('옆집', 0.6473677754402161),
 ('짱아', 0.6224738955497742)]

In [4]:
model.wv.most_similar(positive=['서울','일본'], negative=['한국'])

[('도쿄', 0.6553004384040833),
 ('오사카', 0.6442416906356812),
 ('교토', 0.6120120286941528),
 ('경성부', 0.5605552196502686),
 ('서울특별시', 0.5367318391799927),
 ('종로', 0.5275624990463257),
 ('동대문', 0.5270220041275024),
 ('경성', 0.518776535987854),
 ('간토', 0.5171927213668823),
 ('도쿄도', 0.5089890956878662)]

In [5]:
model.wv.most_similar(positive=['서울','중국'], negative=['한국'])

[('북경', 0.688490629196167),
 ('베이징', 0.6430553197860718),
 ('운남성', 0.6356074810028076),
 ('절강성', 0.6335686445236206),
 ('봉천', 0.6251154541969299),
 ('北京', 0.6066190600395203),
 ('충칭', 0.6049784421920776),
 ('상하이', 0.6012908220291138),
 ('산동성', 0.6002797484397888),
 ('남경', 0.5982213020324707)]

In [6]:
model.wv.most_similar(positive=['오른쪽','남자'], negative=['왼쪽'])

[('여자', 0.7491105198860168),
 ('어른', 0.5937405228614807),
 ('여자아이', 0.5855332612991333),
 ('남녀', 0.5672550201416016),
 ('남자에게', 0.5349474549293518),
 ('아가씨', 0.5177903771400452),
 ('여자도', 0.5176277160644531),
 ('그남자', 0.5144674181938171),
 ('女子', 0.5132307410240173),
 ('외로워', 0.511895477771759)]

In [7]:
model.wv.most_similar(positive=['서울','맛집'])

[('강남', 0.7033297419548035),
 ('동대문', 0.6378874778747559),
 ('평창동', 0.6323271989822388),
 ('이태원', 0.6287323236465454),
 ('여의도', 0.6166844367980957),
 ('연희동', 0.6138433218002319),
 ('강북', 0.6127862334251404),
 ('송파', 0.6107890009880066),
 ('광화문', 0.6103876233100891),
 ('인사동', 0.6073919534683228)]

In [8]:
model.wv['고양이']

array([ 5.4394007 , -0.01980883,  1.8977295 , -1.2494247 , -0.57004064,
        2.567712  , -1.6200314 ,  1.2227641 ,  0.06029971, -1.7774072 ,
       -0.83413273, -0.42919734, -1.2309037 , -1.1024537 , -0.8061239 ,
        0.60492235,  1.8388214 , -1.2539688 , -1.9162188 , -1.8057042 ,
       -0.394243  , -0.02844156, -0.9789619 , -0.11277597, -2.4982111 ,
        0.04807808,  0.77322745, -1.721143  ,  1.2295259 ,  1.2636814 ,
        3.6482937 ,  0.02867755, -3.1084127 ,  0.18008067,  0.42465106,
        0.88798946,  2.722328  , -0.13288637,  0.12018509, -3.4317496 ,
       -1.8254056 ,  2.2829404 ,  1.220089  ,  1.7054838 ,  0.6809032 ,
       -1.1232904 ,  2.914248  , -1.3247129 ,  1.7813191 ,  1.9318762 ,
        0.7911475 , -2.7863338 , -0.8349737 , -1.3192891 , -2.8759706 ,
        1.0561261 ,  2.2265365 ,  0.47400182, -0.14831318, -1.8736936 ,
       -1.2840827 , -0.44321913, -2.437037  , -0.8726344 , -1.0094637 ,
        1.2541952 , -0.12605311, -1.9677713 , -0.8863553 ,  1.45

In [7]:
## 베이지안 필터 사용하기
import math, sys
from konlpy.tag import Twitter

class BayesianFilter:
    """베이지안 필터"""
    def __init__(self):
        self.words = set()      # 출현 단어 기록
        self.word_dict = {}     # 카테고리마다의 출현 횟수를 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
        
    def split(self, text):
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        malist = twitter.pos(text,norm=True,stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외
            if not word[1] in ['Josa','Eomi','Punctuation']:
                results.append(word[0])
        return results
    
    # 단어와 카테고리의 출현 횟수 세기
    def inc_word(self, word, category):
        # 단어를 카테고리에 추가하기
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
        
    def inc_category(self, category):
        # 카테고리 계산하기
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
        
    # 텍스트 학습하기
    def fit(self, text, category):
        """텍스트 학습"""
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
    
    # 단어 리스트에 점수 매기기
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    # 에측하기
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category,score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
    
    # 카테고리 내부의 단어 출현 비율 계산
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d
        

In [9]:
# from bayes import BayesianFilter   => 모듈을 호출하여 BaysianFilter를 로드
bf = BayesianFilter()
# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30%할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현대 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식","광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고", "중요")
bf.fit("계약 잘 부탁드립니다.", "중요")
bf.fit("회의 일정이 등록되었습니다.", "중요")
bf.fit("오늘 일정이 없습니다.", "중요")

# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 = ", pre)
print(scorelist)

결과 =  광고
[('광고', -19.16807495974667), ('중요', -20.449365773467083)]


In [ ]:
import os, glob, json

root_dir = './newstext'
dic_file = root_dir + "/word-dic.json"
data_file = root_dir + '/data.json'
data_file_min = root_dir + '/data-mini.json'

# 어구를 자르고 ID로 변환하기
word_dic = {"_MAX" : 0}
def text_to_ids(text):
    text = text.strip()
    words = text.split(" ")
    result = []
    for n in words:
        n = n.strip()
        if n == "": continue
        if not n in word_dic:
            wid = word_dic[n] = word_dic["_MAX"]
            word_dic["_MAX"] += 1
            print(wid, n)
        else:
            wid = word_dic[n]
        result.append(wid)
    return result
# 파일을 읽고 고정 길이의 배열 리턴하기 (※2)
def file_to_ids(fname):
    with open(fname, "r",encoding='utf-8') as f:
        text = f.read()
        return text_to_ids(text)
# 딕셔너리에 단어 모두 등록하기 (3)
def register_dic():
    files = glob.glob(root_dir+"/*/*.wakati", recursive=True)
    for i in files:
        file_to_ids(i)
# 파일 내부의 단어 세기 (X4)
def count_file_freq(fname):
    cnt = [0 for n in range(word_dic["_MAX"])]
    with open(fname, "r",encoding='utf-8') as f:
        text = f.read().strip()
        ids = text_to_ids(text)
        for wid in ids:
            cnt[wid] += 1
    return cnt
# 카테고리마다 파일 읽어 들이기 (5)
def count_freq(limit = 0):
    X = []
    Y = []
    max_words = word_dic["_MAX"]
    cat_names = []
    for cat in os.listdir(root_dir):
        cat_dir = root_dir + "/" + cat
        if not os.path.isdir(cat_dir): continue
        cat_idx = len(cat_names)
        cat_names.append(cat)
        files = glob.glob(cat_dir+"/*.wakati")
        i = 0
        for path in files:
            print(path)
            cnt = count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0:
                if i > limit: break
                i += 1
    return X,Y
# 단어 딕셔너리 만들기 (※5)
if os.path.exists(dic_file):
    word_dic = json.load(open(dic_file))
else:
    register_dic()
    json.dump(word_dic, open(dic_file, "w",encoding='utf-8'))
# 벡터를 파일로 출력하기 (※6)
# 테스트 목적의 소규모 데이터 만들기
X, Y = count_freq(20)
json.dump({"X": X, "Y": Y}, open(data_file_min,"w",encoding='utf-8'))
# 전체 데이터를 기반으로 데이터 만들기
X, Y = count_freq()
json.dump({"X": X, "Y": Y}, open(data_file, "w",encoding='utf-8'))
print("ok")

In [ ]:
# MLP로 텍스트 분류하기
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import json
import numpy as np

max_words = 56681   # 입력단어수: word-dic.json참고
nb_classes = 6      # 9개의 카테고리

batch_size = 64
nb_epoch = 5

# MLP 모델 생성하기
def build_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# 데이터 읽어 들이기
# data = json.load(open('./data_text/data-mini.json'))
data = json.load(open('./newstext/data.json'))
X = data['X']   # 텍스트를 나타내는 데이터
Y = data['Y']   # 카테고리 데이터


# 학습하기
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
Y_train = np_utils.to_categorical(Y_train, nb_classes)
print(len(X_train),len(Y_train))

model = KerasClassifier(
    build_fn=build_model,
    epochs=nb_epoch,
    batch_size=batch_size
)
model.fit(X_train, Y_train)

# 예측하기
y = model.predict(X_test)
ac_score = metrics.accuracy_score(Y_test, y)
cl_report = metrics.classification_report(Y_test, y)
print('정답률 = ',ac_score)
print('리포트 =\n',cl_report)

In [16]:
def calc_distance(a,b):
    """레벤슈타인 거리 계산하기"""
    if a == b: return 0
    a_len = len(a)
    b_len = len(b)
    if a == "": return b_len
    if b == "": return a_len
    # 2차원 표 (a_len+1, b_len+1) 준비하기
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1):    # 0으로 초기화
        matrix[i] = [0 for j in range(b_len+1)]
    # 0일 때 초깃값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    # 표 채우기
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac==bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1, # 문자삽입
                matrix[i][j-1] + 1, # 문자제거
                matrix[i-1][j-1] + cost # 문자변경
            ])
    return matrix[a_len][b_len]

# "가나다라"와 "가마바라" 의 거리
print(calc_distance("가나다라","가마바라"))

# 실행 예
samples = ["신촌역", "신천군", "신천역", "신발", "마곡역"]
base = samples[0]
r = sorted(samples, key=lambda n: calc_distance(base,n))
for n in r:
    print(calc_distance(base,n),n)

2
0 신촌역
1 신천역
2 신천군
2 신발
2 마곡역
